In [1]:
# =========================
# chargement de donnees
# =========================
import os
from zipfile import ZipFile
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from google.colab import drive

# 1) Monter Google Drive
drive.mount('/content/drive')

# 2) Chemin du zip
zip_path = "/content/drive/MyDrive/Colab Notebooks/COVID-19_Radiography_dataset.zip"
extract_path = "/content/COVID-19_Radiography_Dataset"

# 3) Dézipper si besoin
if not os.path.exists(extract_path):
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/")



Mounted at /content/drive


In [2]:
# =========================
# Réorganisation (2 classes)
# =========================
import shutil

prepared_data_dir = "/content/covid_data_prepared"
os.makedirs(prepared_data_dir, exist_ok=True)


classes = ["COVID", "Normal"]

def find_src_dir(cls):
    
    candidates = [
        os.path.join("/content/COVID-19_Radiography_Dataset", cls, "images"),
        os.path.join("/content/COVID-19_Radiography_Dataset", cls),
        os.path.join("/content", cls, "images"),
        os.path.join("/content", cls),
    ]
    for p in candidates:
        if os.path.isdir(p):
            return p
    raise FileNotFoundError(f"Dossier images introuvable pour la classe '{cls}'.")

copied_counts = {}
for cls in classes:
    src_img_dir = find_src_dir(cls)
    dst_class_dir = os.path.join(prepared_data_dir, cls.replace(" ", "_"))
    os.makedirs(dst_class_dir, exist_ok=True)

    
    exts = {".png", ".jpg", ".jpeg", ".bmp"}
    n = 0
    for filename in os.listdir(src_img_dir):
        if os.path.splitext(filename.lower())[1] in exts:
            shutil.copy(os.path.join(src_img_dir, filename),
                        os.path.join(dst_class_dir, filename))
            n += 1
    copied_counts[cls] = n

print("✅ Réorganisation terminée. Structure ImageFolder prête.")
print("Comptes copiés:", copied_counts)

✅ Réorganisation terminée. Structure ImageFolder prête.
Comptes copiés: {'COVID': 3616, 'Normal': 10192}


In [3]:
# =========================
# Paramètres globaux, Seeds & Device
# =========================
import os, random, numpy as np, torch

# -- chemins --
prepared_data_dir = "/content/covid_data_prepared"

# -- hyper généraux --
SEED = 42
IMG_SIZE = 224
BATCH_TRAIN = 32
BATCH_EVAL  = 128
WORKERS = 2   # ajuste (0–2) selon ta machine

# -- reproductibilité & perf --
def set_seeds(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed)

set_seeds()

DEVICE = torch.device("cpu")
print("Device:", DEVICE)


Device: cpu


In [4]:
# =========================
# Bloc 4 — Réduction équilibrée + splits (identique baseline)
# =========================


PER_CLASS = 500      
VAL_FRAC  = 0.10      
TEST_FRAC = 0.20      
rng = np.random.RandomState(SEED)

base = datasets.ImageFolder(prepared_data_dir)
print("Classes :", base.classes)
print("Mapping :", base.class_to_idx)  

POS_NAME = "COVID"
POS_IDX  = base.class_to_idx[POS_NAME]   
print("Index classe positive (COVID) =", POS_IDX)



name_to_idx = base.class_to_idx
KEEP_NAMES = ["COVID", "Normal"]
KEEP_IDX = [name_to_idx[n] for n in KEEP_NAMES]


idxs_by_class = {ci: [] for ci in KEEP_IDX}
for i, (_, y) in enumerate(base.samples):
    if y in KEEP_IDX:
        idxs_by_class[y].append(i)

# réduction équilibrée
kept_by_class = {}
for c, idxs in idxs_by_class.items():
    idxs = np.array(idxs); rng.shuffle(idxs)
    k = min(PER_CLASS, len(idxs))
    kept_by_class[c] = idxs[:k]

print("Après réduction :",
      {base.classes[c]: len(kept_by_class[c]) for c in KEEP_IDX},
      "Total:", sum(len(v) for v in kept_by_class.values()))

# split stratifié sur les indices RÉDUITS
train_idx, val_idx, test_idx = [], [], []
for c in KEEP_IDX:
    idxs = kept_by_class[c].copy(); rng.shuffle(idxs)
    n = len(idxs)
    n_test = int(round(n * TEST_FRAC))
    n_val  = int(round((n - n_test) * VAL_FRAC))
    test_idx.extend(idxs[:n_test].tolist())
    val_idx.extend(idxs[n_test:n_test+n_val].tolist())
    train_idx.extend(idxs[n_test+n_val:].tolist())

print(f"Splits -> train:{len(train_idx)} | val:{len(val_idx)} | test:{len(test_idx)}")



Classes : ['COVID', 'Normal']
Mapping : {'COVID': 0, 'Normal': 1}
Index classe positive (COVID) = 0
Après réduction : {'COVID': 500, 'Normal': 500} Total: 1000
Splits -> train:720 | val:80 | test:200


In [5]:
# =========================
# Bloc 5 — DataLoaders (identique baseline)
# =========================
from torch.utils.data import Subset, DataLoader

MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])
eval_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

attack_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

base_train  = datasets.ImageFolder(prepared_data_dir, transform=train_tfms)
base_eval   = datasets.ImageFolder(prepared_data_dir, transform=eval_tfms)
base_attack = datasets.ImageFolder(prepared_data_dir, transform=attack_tfms)

train_ds       = Subset(base_train,  train_idx)
val_ds         = Subset(base_eval,   val_idx)
test_ds        = Subset(base_eval,   test_idx)
attack_test_ds = Subset(base_attack, test_idx)

loader_kwargs = dict(pin_memory=(DEVICE.type=="cuda"))
if WORKERS > 0:
    loader_kwargs.update(num_workers=WORKERS, persistent_workers=True, prefetch_factor=2)

train_loader       = DataLoader(train_ds,       batch_size=BATCH_TRAIN, shuffle=True,  **loader_kwargs)
val_loader         = DataLoader(val_ds,         batch_size=BATCH_EVAL,  shuffle=False, **loader_kwargs)
test_loader        = DataLoader(test_ds,        batch_size=BATCH_EVAL,  shuffle=False, **loader_kwargs)
attack_test_loader = DataLoader(attack_test_ds, batch_size=BATCH_EVAL,  shuffle=False, **loader_kwargs)

print("✅ DataLoaders prêts.")


✅ DataLoaders prêts.


In [6]:
# =========================
# Bloc 6 — Modèle CNN (identique baseline)
# =========================
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2, dropout=0.3):
        super().__init__()
        self.b1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True), nn.MaxPool2d(2)
        )
        self.b2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True), nn.MaxPool2d(2)
        )
        self.b3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128), nn.ReLU(inplace=True), nn.MaxPool2d(2)
        )
        self.gap  = nn.AdaptiveAvgPool2d((1,1))
        self.drop = nn.Dropout(dropout)
        self.fc   = nn.Linear(128, num_classes)
        # init propre
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight); nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.b1(x); x = self.b2(x); x = self.b3(x)
        x = self.gap(x); x = torch.flatten(x, 1)
        x = self.drop(x); x = self.fc(x)
        return x

    def extract_features(self, x):
        x = self.b1(x); x = self.b2(x); x = self.b3(x)
        x = self.gap(x); return torch.flatten(x, 1)

model = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE)




In [7]:
!pip -q install adversarial-robustness-toolbox==1.17.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.3 MB/s eta 0:00:00


In [8]:
# =========================
# Bloc 7 — Entraînement TEACHER (CE standard, CPU-only)
# =========================
import time, copy, torch
import torch.nn as nn

teacher = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE) 

criterion_ce = nn.CrossEntropyLoss()
opt_t = torch.optim.AdamW(teacher.parameters(), lr=1e-3, weight_decay=1e-4)
EPOCHS_T, PATIENCE_T = 25, 5
best_val, no_impr = float("inf"), 0
best_state_t = copy.deepcopy(teacher.state_dict())

def evaluate_loss_acc(model, loader):
    model.eval(); loss_sum = 0.0; n = 0; correct = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE); y = y.to(DEVICE)
            logits = model(x)
            loss = criterion_ce(logits, y)
            loss_sum += loss.item() * y.size(0); n += y.size(0)
            correct += (logits.argmax(1) == y).sum().item()
    return loss_sum / n, correct / n

for epoch in range(1, EPOCHS_T + 1):
    teacher.train(); t0 = time.time(); run_loss = 0.0; n = 0
    for x, y in train_loader:
        x = x.to(DEVICE); y = y.to(DEVICE)
        opt_t.zero_grad(set_to_none=True)
        logits = teacher(x)
        loss = criterion_ce(logits, y)
        loss.backward()
        opt_t.step()
        run_loss += loss.item() * y.size(0); n += y.size(0)

    train_loss = run_loss / n
    val_loss, val_acc = evaluate_loss_acc(teacher, val_loader)
    print(f"[Teacher] Epoch {epoch:02d} | train_loss={train_loss:.4f} | "
          f"val_loss={val_loss:.4f} | val_acc={val_acc:.3f} | {time.time()-t0:.1f}s")

    if val_loss < best_val - 1e-4:
        best_val = val_loss; no_impr = 0
        best_state_t = copy.deepcopy(teacher.state_dict())
    else:
        no_impr += 1
        if no_impr >= PATIENCE_T:
            print("Early stopping (teacher)."); break

teacher.load_state_dict(best_state_t)
teacher.eval()
print("✅ Teacher prêt (CPU).")


[Teacher] Epoch 01 | train_loss=0.6581 | val_loss=0.7097 | val_acc=0.650 | 128.3s
[Teacher] Epoch 02 | train_loss=0.5987 | val_loss=0.6764 | val_acc=0.662 | 110.9s
[Teacher] Epoch 03 | train_loss=0.6091 | val_loss=0.7409 | val_acc=0.613 | 111.2s
[Teacher] Epoch 04 | train_loss=0.5857 | val_loss=0.6430 | val_acc=0.675 | 118.9s
[Teacher] Epoch 05 | train_loss=0.5869 | val_loss=0.6084 | val_acc=0.675 | 112.9s
[Teacher] Epoch 06 | train_loss=0.5545 | val_loss=0.5962 | val_acc=0.787 | 113.9s
[Teacher] Epoch 07 | train_loss=0.5809 | val_loss=0.5788 | val_acc=0.700 | 113.8s
[Teacher] Epoch 08 | train_loss=0.5430 | val_loss=0.6449 | val_acc=0.675 | 117.2s
[Teacher] Epoch 09 | train_loss=0.5427 | val_loss=0.5740 | val_acc=0.725 | 114.3s
[Teacher] Epoch 10 | train_loss=0.5099 | val_loss=0.5322 | val_acc=0.775 | 123.7s
[Teacher] Epoch 11 | train_loss=0.5181 | val_loss=0.6163 | val_acc=0.738 | 114.4s
[Teacher] Epoch 12 | train_loss=0.5323 | val_loss=0.5721 | val_acc=0.700 | 115.4s
[Teacher] Epoch 

In [9]:
# =========================
# Bloc 8 — Entraînement STUDENT par distillation
# =========================
import time, copy
import torch
import torch.nn as nn

# Hyper distillation
T       = 8.0          
ALPHA   = 0.7         
LR_S    = 1e-3
EPOCHS_S, PATIENCE_S = 25, 5

student = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE)
opt_s   = torch.optim.AdamW(student.parameters(), lr=LR_S, weight_decay=1e-4)
kldiv   = nn.KLDivLoss(reduction="batchmean")
criterion_ce = nn.CrossEntropyLoss()

best_val_s, no_impr_s = float("inf"), 0
best_state_s = copy.deepcopy(student.state_dict())

# figer le teacher
teacher.eval()
for p in teacher.parameters():
    p.requires_grad_(False)

def distill_loss(student_logits, teacher_logits, y_true, T=T, alpha=ALPHA):
    # soft targets du teacher à T (sans gradient)
    with torch.no_grad():
        p_teacher = torch.softmax(teacher_logits / T, dim=1)
    # student à T
    log_p_student = torch.log_softmax(student_logits / T, dim=1)
    loss_distill = (T * T) * kldiv(log_p_student, p_teacher)
    loss_hard    = criterion_ce(student_logits, y_true)
    return alpha * loss_distill + (1.0 - alpha) * loss_hard

def evaluate_student(model, loader):
    model.eval(); loss_sum=0.0; n=0; correct=0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE); y = y.to(DEVICE)
            logits = model(x); loss = criterion_ce(logits, y)
            loss_sum += loss.item() * y.size(0); n += y.size(0)
            correct  += (logits.argmax(1) == y).sum().item()
    return loss_sum / n, correct / n

for epoch in range(1, EPOCHS_S + 1):
    student.train(); t0 = time.time(); run_loss = 0.0; n = 0
    for x, y in train_loader:
        x = x.to(DEVICE); y = y.to(DEVICE)

        # forward teacher (no grad) sur le même batch (avec aug du train_loader)
        with torch.no_grad():
            teacher_logits = teacher(x)

        opt_s.zero_grad(set_to_none=True)
        student_logits = student(x)
        loss = distill_loss(student_logits, teacher_logits, y)
        loss.backward(); opt_s.step()

        run_loss += loss.item() * y.size(0); n += y.size(0)

    train_loss = run_loss / n
    val_loss, val_acc = evaluate_student(student, val_loader)
    print(f"[Student] Epoch {epoch:02d} | train_loss={train_loss:.4f} | "
          f"val_loss={val_loss:.4f} | val_acc={val_acc:.3f} | {time.time()-t0:.1f}s")

    if val_loss < best_val_s - 1e-4:
        best_val_s = val_loss; no_impr_s = 0
        best_state_s = copy.deepcopy(student.state_dict())
    else:
        no_impr_s += 1
        if no_impr_s >= PATIENCE_S:
            print("Early stopping (student)."); break

student.load_state_dict(best_state_s)
student.eval()
print("✅ Student prêt.")


[Student] Epoch 01 | train_loss=0.3015 | val_loss=0.6780 | val_acc=0.613 | 170.0s
[Student] Epoch 02 | train_loss=0.2346 | val_loss=0.6377 | val_acc=0.662 | 164.4s
[Student] Epoch 03 | train_loss=0.2340 | val_loss=0.6352 | val_acc=0.625 | 158.5s
[Student] Epoch 04 | train_loss=0.2285 | val_loss=0.6507 | val_acc=0.650 | 168.0s
[Student] Epoch 05 | train_loss=0.2135 | val_loss=0.6244 | val_acc=0.650 | 158.5s
[Student] Epoch 06 | train_loss=0.2143 | val_loss=0.5836 | val_acc=0.700 | 158.2s
[Student] Epoch 07 | train_loss=0.2128 | val_loss=0.5729 | val_acc=0.688 | 169.7s
[Student] Epoch 08 | train_loss=0.2008 | val_loss=0.5669 | val_acc=0.688 | 164.7s
[Student] Epoch 09 | train_loss=0.1948 | val_loss=0.5752 | val_acc=0.675 | 162.1s
[Student] Epoch 10 | train_loss=0.1904 | val_loss=0.5649 | val_acc=0.762 | 160.2s
[Student] Epoch 11 | train_loss=0.2039 | val_loss=0.5636 | val_acc=0.700 | 162.1s
[Student] Epoch 12 | train_loss=0.1810 | val_loss=0.5625 | val_acc=0.750 | 162.4s
[Student] Epoch 

In [10]:
# =========================
# Bloc 9 — Évaluation clean du STUDENT (CPU-only, COVID = classe positive)
# =========================
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

student.eval()
y_true, y_pred, y_prob = [], [], []
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)                         
        logits = student(x)
        prob_pos = torch.softmax(logits, dim=1)[:, POS_IDX].cpu().numpy()  
        y_prob.extend(prob_pos)
        y_pred.extend(logits.argmax(1).cpu().numpy())
        y_true.extend(y.numpy())


y_true_pos = [1 if t == POS_IDX else 0 for t in y_true]

print("DEF(Distill) Test | acc:{:.4f} | prec:{:.4f} | rec:{:.4f} | f1:{:.4f} | auc:{:.4f}".format(
    accuracy_score(y_true, y_pred),
    precision_score(y_true, y_pred, pos_label=POS_IDX, zero_division=0),
    recall_score(y_true, y_pred,    pos_label=POS_IDX, zero_division=0),
    f1_score(y_true, y_pred,        pos_label=POS_IDX, zero_division=0),
    roc_auc_score(y_true_pos, y_prob),
))

# Pour la suite des attaques, on désigne 'model' = student
model = student


DEF(Distill) Test | acc:0.7700 | prec:0.7077 | rec:0.9200 | f1:0.8000 | auc:0.8094


In [11]:
# =========================
# Bloc 10 — Prépa attaques 
# =========================
import numpy as np, torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def dataloader_to_numpy(dl):
    xs, ys = [], []
    for x, y in dl:
        xs.append(x.numpy())   
        ys.append(y.numpy())
    return np.concatenate(xs, 0), np.concatenate(ys, 0)

X_test_np, y_test_np = dataloader_to_numpy(attack_test_loader)

# Tenseurs de normalisation
MEAN_T = torch.tensor([0.485, 0.456, 0.406], device=DEVICE).view(1, 3, 1, 1)
STD_T  = torch.tensor([0.229, 0.224, 0.225], device=DEVICE).view(1, 3, 1, 1)

def eval_np_batchwise(X_np, y_np, batch=BATCH_EVAL, pos_idx=POS_IDX):
    """
    Évalue le modèle sur des entrées en [0,1] (on applique Normalize ici).
    pos_idx = index de la classe positive (COVID).
    """
    y_true, y_pred, y_prob = [], [], []
    model.eval()
    with torch.no_grad():
        for i in range(0, len(X_np), batch):
            xt = torch.from_numpy(X_np[i:i+batch]).to(DEVICE).float()  
            xt = (xt - MEAN_T) / STD_T                                
            logits = model(xt)
            prob_pos = torch.softmax(logits, dim=1)[:, pos_idx].cpu().numpy()
            pred     = logits.argmax(1).cpu().numpy()
            y_prob.extend(prob_pos)
            y_pred.extend(pred)
            y_true.extend(y_np[i:i+batch])

    
    y_true_pos = [1 if t == pos_idx else 0 for t in y_true]

    return dict(
        acc=accuracy_score(y_true, y_pred),
        precision=precision_score(y_true, y_pred, pos_label=pos_idx, zero_division=0),
        recall=recall_score(y_true, y_pred,    pos_label=pos_idx, zero_division=0),
        f1=f1_score(y_true, y_pred,            pos_label=pos_idx, zero_division=0),
        auc=roc_auc_score(y_true_pos, y_prob),
    )


In [12]:
# =========================
# Bloc 11 — FGSM / PGD / BIM 
# =========================
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, BasicIterativeMethod
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 1) Classifier ART 
clf = PyTorchClassifier(
    model=model,
    loss=nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-3),
    input_shape=(3, IMG_SIZE, IMG_SIZE),
    nb_classes=2,
    clip_values=(0.0, 1.0),
    preprocessing=(np.array(MEAN, dtype=np.float32), np.array(STD, dtype=np.float32)),
    device_type="cpu",
)

# 2) Attaques en streaming
PS_GRID = [2/255, 4/255, 8/255]
results_fpb = []

def eval_adv_stream(attack, X_np, y_np, batch=BATCH_EVAL, pos_idx=POS_IDX):
    """
    Génère X_adv en [0,1] avec ART, applique Normalize, puis évalue.
    pos_idx = index de la classe positive (COVID).
    """
    y_true, y_pred, y_prob = [], [], []
    model.eval()
    for i in range(0, len(X_np), batch):
        Xb = X_np[i:i+batch]; yb = y_np[i:i+batch]
        Xab = attack.generate(Xb)  
        with torch.no_grad():
            xt = torch.from_numpy(Xab).to(DEVICE).float()  
            xt = (xt - MEAN_T) / STD_T                    
            logits   = model(xt)
            prob_pos = torch.softmax(logits, dim=1)[:, pos_idx].cpu().numpy()
            pred     = logits.argmax(1).cpu().numpy()
        y_true.extend(yb); y_pred.extend(pred); y_prob.extend(prob_pos)

    y_true_pos = [1 if t == pos_idx else 0 for t in y_true]  

    return dict(
        acc      = accuracy_score(y_true, y_pred),
        precision= precision_score(y_true, y_pred, pos_label=pos_idx, zero_division=0),
        recall   = recall_score(y_true, y_pred,    pos_label=pos_idx, zero_division=0),
        f1       = f1_score(y_true, y_pred,        pos_label=pos_idx, zero_division=0),
        auc      = roc_auc_score(y_true_pos, y_prob),
    )

# FGSM
for eps in PS_GRID:
    atk = FastGradientMethod(estimator=clf, eps=eps, batch_size=BATCH_EVAL)
    m = eval_adv_stream(atk, X_test_np, y_test_np, batch=BATCH_EVAL, pos_idx=POS_IDX)
    results_fpb.append(dict(attack="FGSM", eps=eps, step=None, iters=1, **m))
    print(f"FGSM eps={eps:.5f} | acc={m['acc']:.4f} | prec={m['precision']:.4f} | rec={m['recall']:.4f} "
          f"| f1={m['f1']:.4f} | auc={m['auc']:.4f}")

# PGD 
for eps in PS_GRID:
    atk = ProjectedGradientDescent(
        estimator=clf, eps=eps, eps_step=eps/8, max_iter=20,
        batch_size=BATCH_EVAL, targeted=False, num_random_init=0
    )
    m = eval_adv_stream(atk, X_test_np, y_test_np, batch=BATCH_EVAL, pos_idx=POS_IDX)
    results_fpb.append(dict(attack="PGD", eps=eps, step=eps/8, iters=20, random_init=False, **m))
    print(f"PGD  eps={eps:.5f} | acc={m['acc']:.4f} | prec={m['precision']:.4f} | rec={m['recall']:.4f} "
          f"| f1={m['f1']:.4f} | auc={m['auc']:.4f}")

# BIM
for eps in PS_GRID:
    atk = BasicIterativeMethod(estimator=clf, eps=eps, eps_step=eps/12, max_iter=12, batch_size=BATCH_EVAL)
    m = eval_adv_stream(atk, X_test_np, y_test_np, batch=BATCH_EVAL, pos_idx=POS_IDX)
    results_fpb.append(dict(attack="BIM", eps=eps, step=eps/12, iters=12, **m))
    print(f"BIM  eps={eps:.5f} | acc={m['acc']:.4f} | prec={m['precision']:.4f} | rec={m['recall']:.4f} "
          f"| f1={m['f1']:.4f} | auc={m['auc']:.4f}")


FGSM eps=0.00784 | acc=0.5150 | prec=0.5102 | rec=0.7500 | f1=0.6073 | auc=0.5321
FGSM eps=0.01569 | acc=0.4600 | prec=0.4718 | rec=0.6700 | f1=0.5537 | auc=0.3769
FGSM eps=0.03137 | acc=0.4750 | prec=0.4865 | rec=0.9000 | f1=0.6316 | auc=0.2871


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD  eps=0.00784 | acc=0.3050 | prec=0.2990 | rec=0.2900 | f1=0.2944 | auc=0.2831


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

BIM  eps=0.01569 | acc=0.2250 | prec=0.1333 | rec=0.1000 | f1=0.1143 | auc=0.2717


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

BIM  eps=0.03137 | acc=0.2300 | prec=0.1143 | rec=0.0800 | f1=0.0941 | auc=0.2773


In [13]:
# =========================
# (Option) Bloc 12 — C&W (à lancer si besoin, coûteux) — CPU-only
# =========================
from art.attacks.evasion import CarliniL2Method

CONF = 0
BATCH_ATTACK = 32
results_cw = []


atk = CarliniL2Method(
    classifier=clf,
    targeted=False,
    confidence=CONF,
    max_iter=75,
    binary_search_steps=1,
    learning_rate=0.02,
    initial_const=0.3,
    batch_size=BATCH_ATTACK
)


m_cw = eval_adv_stream(atk, X_test_np, y_test_np, batch=BATCH_ATTACK, pos_idx=POS_IDX)
results_cw.append(dict(
    attack="CW-L2", confidence=CONF, max_iter=75, bsearch=1, c0=0.3, **m_cw
))

print(
    f"C&W c0=0.3 | acc={m_cw['acc']:.4f} | prec={m_cw['precision']:.4f} "
    f"| rec={m_cw['recall']:.4f} | f1={m_cw['f1']:.4f} | auc={m_cw['auc']:.4f}"
)


C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

C&W c0=0.3 | acc=0.7350 | prec=0.6880 | rec=0.8600 | f1=0.7644 | auc=0.8065
